In [228]:
import pandas as pd
from pymongo import MongoClient
import requests
from dotenv import load_dotenv
load_dotenv()
import numpy as np

# 1) Cleanse

In [2]:
client = MongoClient("mongodb://localhost:27017/")

In [3]:
db = client.companies 

In [4]:
companies = db.companies.find()

In [5]:
data_companies = pd.DataFrame(companies)

In [6]:
data_companies.head(3)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,video_embeds,screenshots,external_links,partners,founded_month,founded_day,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,"[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],9.0,15.0,NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],10.0,17.0,NaN,NaN,NaN,NaN


In [7]:
#esto irá con mis propios criterios: founded year 2003??????????????????????????????????????
companies = db.companies.find({"$and":[
                                            {"offices": {"$exists": True}},
                                            {"offices": {"$ne": None}}, 
                                            {"$and" : [{"offices.latitude": {"$exists": True}}, {"offices.longitude": {"$exists": True}}]},
                                            {"category_code": {"$exists": True}},
                                            {"category_code": {"$ne": None}}, 
                                            {"founded_year": {"$exists": True}}, 
                                            {"founded_year": {"$gte": 2003}},
                                            {"deadpooled_year": None},
                                            {"number_of_employees": {"$exists": True}},
                                            {"number_of_employees": {"$gte": 10}},
                                            {"total_money_raised": {"$exists": True}},
                                            {"total_money_raised":{"$ne":None}},
                                            {"total_money_raised": {"$not":{"$size":0}}}, 
                                            
                                            ]
                                            },
                                            {"_id": 0, "crunchbase_url": 0, "products": 0, 
                                             "acquisition": 0, "acquisitions": 0, "video_embeds": 0, 
                                             "screenshots": 0, "external_links": 0, "partners": 0, 
                                             "image": 0, 
                                             'deadpooled_day': 0, 'deadpooled_month': 0, 
                                             'deadpooled_url': 0, 'deadpooled_year': 0,
                                            }
                                     )


In [8]:
companies = pd.DataFrame(companies)
offices = companies.set_index('name').offices.apply(pd.Series).stack().reset_index(level=-1, drop=True).reset_index()
offices.columns = ['name', 'office']
companies = companies.drop('offices', 1)
companies = companies.merge(offices, on='name', how='outer')

In [15]:
companies.head(3)

,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,...,providerships,total_money_raised,funding_rounds,investments,milestones,ipo,office,lat,long,position
0,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,6.0,...,[],$16.5M,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",[],"[{'id': 15460, 'description': 'Announced hire ...",None,"{'description': 'Headquarters', 'address1': '9...",34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,Digg,digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60,2004,10.0,...,"[{'title': 'Public Relations', 'is_past': True...",$45M,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...",[],"[{'id': 9588, 'description': 'Another Digg Exe...",None,"{'description': None, 'address1': '135 Mississ...",37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
2,Twitter,twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300,2006,3.0,...,"[{'title': 'Hosting', 'is_past': True, 'provid...",$1.16B,"[{'id': 261, 'round_code': 'a', 'source_url': ...",[],"[{'id': 1, 'description': 'Survives iPhone 3G ...","{'valuation_amount': 18100000000, 'valuation_c...","{'description': '', 'address1': '1355 Market S...",37.776805,-122.416924,"{'type': 'Point', 'coordinates': [-122.4169244..."


In [16]:
def createLat(office):
    return office['latitude']
def createLon(office):
    return office['longitude']

companies['lat'] = companies['office'].apply(createLat)
companies['long'] = companies['office'].apply(createLon)
companies.dropna(subset=['lat', 'long'], inplace=True)

In [17]:
companies.head(3)

,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,...,providerships,total_money_raised,funding_rounds,investments,milestones,ipo,office,lat,long,position
0,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,6.0,...,[],$16.5M,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",[],"[{'id': 15460, 'description': 'Announced hire ...",None,"{'description': 'Headquarters', 'address1': '9...",34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,Digg,digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60,2004,10.0,...,"[{'title': 'Public Relations', 'is_past': True...",$45M,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...",[],"[{'id': 9588, 'description': 'Another Digg Exe...",None,"{'description': None, 'address1': '135 Mississ...",37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
2,Twitter,twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300,2006,3.0,...,"[{'title': 'Hosting', 'is_past': True, 'provid...",$1.16B,"[{'id': 261, 'round_code': 'a', 'source_url': ...",[],"[{'id': 1, 'description': 'Survives iPhone 3G ...","{'valuation_amount': 18100000000, 'valuation_c...","{'description': '', 'address1': '1355 Market S...",37.776805,-122.416924,"{'type': 'Point', 'coordinates': [-122.4169244..."


In [18]:
def createGeoJson(office):
    return {
        "type":"Point",
        "coordinates":[office["long"],office["lat"]]
    }

companies.dropna(subset=['lat', 'long'], inplace=True)
companies['position'] = companies.apply(createGeoJson, axis=1)
display(companies.head(3))
companies.shape

,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,...,providerships,total_money_raised,funding_rounds,investments,milestones,ipo,office,lat,long,position
0,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,6.0,...,[],$16.5M,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",[],"[{'id': 15460, 'description': 'Announced hire ...",None,"{'description': 'Headquarters', 'address1': '9...",34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,Digg,digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60,2004,10.0,...,"[{'title': 'Public Relations', 'is_past': True...",$45M,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...",[],"[{'id': 9588, 'description': 'Another Digg Exe...",None,"{'description': None, 'address1': '135 Mississ...",37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
2,Twitter,twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300,2006,3.0,...,"[{'title': 'Hosting', 'is_past': True, 'provid...",$1.16B,"[{'id': 261, 'round_code': 'a', 'source_url': ...",[],"[{'id': 1, 'description': 'Survives iPhone 3G ...","{'valuation_amount': 18100000000, 'valuation_c...","{'description': '', 'address1': '1355 Market S...",37.776805,-122.416924,"{'type': 'Point', 'coordinates': [-122.4169244..."


(2283, 31)

In [24]:
import re

def authRequest(url):
    response = requests.get("{}".format(url))
    return response.json()

def currencyconverter(txt):
    txt = [float(re.sub("[^0-9.]", "", txt)), re.sub("[0-9.]", "", txt)]
    abrev = {"M": "million","B": "billion", "kr": "SEK", "k": "thousand", "€": "EUR", "£": "GBP", "C$": "CAD", "$": ""}
    scales = {"billion": 1000000000, "million": 1000000, "thousand": 1000}
    currencies = {"kr": "SEK", "€": "EUR", "£": "GBP", "C$": "CAD"}
    converter = authRequest('https://api.exchangeratesapi.io/latest?base=USD')['rates']
    for k, v in abrev.items():
        txt[1] = txt[1].replace(k, v)
    for k, v in scales.items():
        if k in txt[1]:
            txt[0] = txt[0] * v
    for v in list(currencies.values()):
        if v in txt[1]:
            txt[0] = txt[0]/converter[v]
    return int(txt[0])


14340299


In [26]:
companies['total_money_raised'] = companies['total_money_raised'].apply(currencyconverter)

KeyboardInterrupt: 

In [27]:
display(companies)

,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,...,providerships,total_money_raised,funding_rounds,investments,milestones,ipo,office,lat,long,position
0,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,6.0,...,[],$16.5M,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",[],"[{'id': 15460, 'description': 'Announced hire ...",None,"{'description': 'Headquarters', 'address1': '9...",34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,Digg,digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60,2004,10.0,...,"[{'title': 'Public Relations', 'is_past': True...",$45M,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...",[],"[{'id': 9588, 'description': 'Another Digg Exe...",None,"{'description': None, 'address1': '135 Mississ...",37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
2,Twitter,twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300,2006,3.0,...,"[{'title': 'Hosting', 'is_past': True, 'provid...",$1.16B,"[{'id': 261, 'round_code': 'a', 'source_url': ...",[],"[{'id': 1, 'description': 'Survives iPhone 3G ...","{'valuation_amount': 18100000000, 'valuation_c...","{'description': '', 'address1': '1355 Market S...",37.776805,-122.416924,"{'type': 'Point', 'coordinates': [-122.4169244..."
3,Scribd,scribd,http://scribd.com,http://blog.scribd.com/,http://blog.scribd.com/feed/,scribd,news,50,2007,NaN,...,[],$25.8M,"[{'id': 10, 'round_code': 'seed', 'source_url'...",[],"[{'id': 4960, 'description': 'Scribd launches ...",None,"{'description': 'HQ', 'address1': '539 Bryant ...",37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,..."
4,MeetMoi,meetmoi,http://www.meetmoi.com,,,meetmoi,social,15,2007,NaN,...,[],$5.58M,"[{'id': 18, 'round_code': 'a', 'source_url': '...",[],[],None,"{'description': None, 'address1': '', 'address...",40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ..."
5,Facebook,facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299,2004,2.0,...,"[{'title': '', 'is_past': False, 'provider': {...",$2.43B,"[{'id': 2, 'round_code': 'angel', 'source_url'...","[{'funding_round': {'round_code': 'seed', 'sou...","[{'id': 108, 'description': 'Facebook adds com...","{'valuation_amount': 104000000000, 'valuation_...","{'description': 'Headquarters', 'address1': '1...",37.416050,-122.151801,"{'type': 'Point', 'coordinates': [-122.151801,..."
6,Facebook,facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299,2004,2.0,...,"[{'title': '', 'is_past': False, 'provider': {...",$2.43B,"[{'id': 2, 'round_code': 'angel', 'source_url'...","[{'funding_round': {'round_code': 'seed', 'sou...","[{'id': 108, 'description': 'Facebook adds com...","{'valuation_amount': 104000000000, 'valuation_...","{'description': 'Europe HQ', 'address1': '', '...",53.344104,-6.267494,"{'type': 'Point', 'coordinates': [-6.267494, 5..."
7,Facebook,facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299,2004,2.0,...,"[{'title': '', 'is_past': False, 'provider': {...",$2.43B,"[{'id': 2, 'round_code': 'angel', 'source_url'...","[{'funding_round': {'round_code': 'seed', 'sou...","[{'id': 108, 'description': 'Facebook adds com...","{'valuation_amount': 104000000000, 'valuation_...","{'description': 'New York', 'address1': '340 M...",40.755716,-73.979247,"{'type': 'Point', 'coordinates': [-73.9792469,..."
8,Powerset,powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60,2006,10.0,...,"[{'title': '', 'is_past': False, 'provider': {...",$22.5M,"[{'id': 22, 'round_code': 'a', 'source_url': '...",[],[],None,"{'description': None, 'address1': '475 Brannan...",37.778

In [ ]:
#Importar archivo json a Mongo
# mongoimport --db companies --collection selected --jsonArray offices.json
companies.to_json('./offices.json', orient="records")

# 2) Filtrado 

### 2.1) Importación BD limpia basada en money raised y keywords close companies

In [28]:
#Volverlo a traer a Jupyter
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

def categories_builder(txt):
    regex = ""
    categories = [cat.strip() for cat in txt.split(",")]
    for e in categories:
        regex += "(?i){}|".format(e)
    return regex[:-1]

#cat_regex = categories_builder(input()) --> en realidad no es input, si no parametros
#escribe los keywords que definan a los sectores de esas empresas
#por ahora lo pongo así para ahorrar tiempo y no tener que poner las categorías todo el rato:
cat_regex = "(?i)tech|(?i)web|(?i)design|(?i)code|(?i)mobile|(?i)advertising"

#lo mismo con el capital raised (en dollars), que es en parametros pero por ahora lo dejo puesto fijo:

min_money_raised = 1000000
                   
selected = db.selected.find({"$and" : [{"total_money_raised": {"$gte": min_money_raised}},
                                                {"$or" : [{"tag_list":{"$regex": cat_regex }}, {"category_code":{"$regex": cat_regex}}, {"name":{"$regex": cat_regex}}, {"description":{"$regex": cat_regex}}, {"overview":{"$regex": cat_regex}}]}]})

selected_df = pd.DataFrame(selected)
#selected_df.dropna(subset=['lat', 'long'], inplace=True)

display(selected_df.head(3))
print(selected_df.shape)

,_id,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,...,providerships,total_money_raised,funding_rounds,investments,milestones,ipo,office,lat,long,position
0,5d7e63601ce3a9e7731a4339,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,...,[],16500000,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",[],"[{'id': 15460, 'description': 'Announced hire ...",None,"{'description': 'Headquarters', 'address1': '9...",34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,5d7e63601ce3a9e7731a433a,MeetMoi,meetmoi,http://www.meetmoi.com,,,meetmoi,social,15,2007,...,[],5580000,"[{'id': 18, 'round_code': 'a', 'source_url': '...",[],[],None,"{'description': None, 'address1': '', 'address...",40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ..."
2,5d7e63601ce3a9e7731a433b,Scribd,scribd,http://scribd.com,http://blog.scribd.com/,http://blog.scribd.com/feed/,scribd,news,50,2007,...,[],25800000,"[{'id': 10, 'round_code': 'seed', 'source_url'...",[],"[{'id': 4960, 'description': 'Scribd launches ...",None,"{'description': 'HQ', 'address1': '539 Bryant ...",37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,..."


(863, 32)


### 2.2) Utilizar GeoNear function para, dada una lista de todas las coordenadas, ver cuántas empresas tiene cerca de las que hemos seleccionado 

In [29]:
#funcion para sacar compañías a la redonda
#crear el geoindex en MongoDB Compass
def getCompaniesNear(lat,long, max_meters=2000):
    return list(db.selected.find({
        "position": {
            "$near": {
               "$geometry": {
                  "type": "Point" ,
                  "coordinates": [ long , lat ]
               },
               "$maxDistance": max_meters
             }
       }
    }))


# 1) aqui la base de datos puedo poner la de selected,
# 2) pero longitudes y latitudes coger todas para tener más dónde mirar -- > crear una lista con todos los geopoints
#de companies!

In [60]:
#creo lista de TODAS las coordenadas para probarlas todas y ver cuál tiene más empresas de las de selected alrededor
coord_list = []

for i in range(len(companies.index)):
    coord = companies.iloc[i]["position"]['coordinates']
    coord_list.append([coord,len(pd.DataFrame(getCompaniesNear(coord[1],coord[0])).index)])

coord_list.sort(key = lambda x: x[1], reverse = True)

In [61]:
coord_list

[[[-122.408646, 37.784137], 144],
 [[-122.4013217, 37.7814765], 144],
 [[-122.4041764, 37.7819286], 143],
 [[-122.410943, 37.787683], 141],
 [[-122.403256, 37.78236], 141],
 [[-122.405081, 37.785792], 141],
 [[-122.400591, 37.782163], 141],
 [[-122.402195, 37.786183], 140],
 [[-122.40971, 37.788796], 140],
 [[-122.4041075, 37.7871306], 140],
 [[-122.409173, 37.788482], 139],
 [[-122.4020593, 37.7797255], 139],
 [[-122.406912, 37.781002], 138],
 [[-122.401803, 37.778991], 138],
 [[-122.4084994, 37.7909427], 133],
 [[-73.993182, 40.741179], 129],
 [[-73.9930489, 40.73993], 128],
 [[-73.991561, 40.741276], 128],
 [[-73.991561, 40.741276], 128],
 [[-73.991561, 40.741276], 128],
 [[-73.991561, 40.741276], 128],
 [[-73.993158, 40.739833], 127],
 [[-73.9912208, 40.7413559], 127],
 [[-122.402759, 37.787646], 127],
 [[-73.993127, 40.740337], 126],
 [[-122.404113, 37.789471], 126],
 [[-122.404052, 37.789634], 125],
 [[-73.993376, 40.740441], 125],
 [[-73.993376, 40.740441], 125],
 [[-73.993376, 

In [224]:
import numpy as np

def loc_selector_nearcompanies(lst):
    q = np.quantile([x[1] for x in coord_list], .99)
    return [e for e in lst if e[1] >= q ]

# ESTE 0.90 PODRÍA DEPENDER DE LA IMPORTANCIA QUE LE DE LA PERSONA AL DATASET
# Esto estadísticamente qué significa?

In [225]:
df_coordenates = pd.DataFrame(loc_selector_nearcompanies(coord_list), columns= ['coord', 'near_companies'])
df_coordenates

,coord,near_companies
0,"[-122.408646, 37.784137]",144
1,"[-122.4013217, 37.7814765]",144
2,"[-122.4041764, 37.7819286]",143
3,"[-122.410943, 37.787683]",141
4,"[-122.403256, 37.78236]",141
5,"[-122.405081, 37.785792]",141
6,"[-122.400591, 37.782163]",141
7,"[-122.402195, 37.786183]",140
8,"[-122.40971, 37.788796]",140
9,"[-122.4041075, 37.7871306]",140


# 3) API Google Maps

In [226]:
import time

bego_key = ""

def countGoogledata(place, radius, coord, key):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&keyword={}&key={}".format(str(coord[1]), str(coord[0]), str(radius), place, key)
    data = [(requests.get(url)).json()]
    while "next_page_token" in data[-1]:
        new_url = url+"&pagetoken={}".format(data[-1]['next_page_token'])
        time.sleep(2)
        new_req = (requests.get(str(new_url))).json()
        data.append(new_req)
    total_results = 0
    for x in data:
        total_results += len(x['results'])
    return total_results

#https://developers.google.com/places/web-service/search
#https://developers.google.com/places/web-service/get-api-key
#https://console.cloud.google.com/billing/0163C0-E6306C-B76C83?project=project-ironhack-pymongo
#https://developers.google.com/calendar/v3/pagination

In [227]:
list_count_places = []

for x in df_coordenates['coord']:
    list_count_places.append(countGoogledata("starbucks", 1500, x, bego_key))
    
list_count_places

KeyboardInterrupt: 

# Next

In [ ]:
"""

1) APLICAR EL COUNT GOOGLE DATA A MI DATAFRAME
     --> Duda, tarda mucho!
            - Estoy haciendo algo mal
            - Tengo que coger menos filas en el dataset (son demasiadas?)
            
2) PONER TODO EN FUNCIONES Y PASAR A VISUAL CODE

3) CONECTAR A OTRAS APIs

4) HACER UN RANKING DE LOS DISTINTOS SITIOS (en base a la importancia preguntada al usuario)

5) PIPELINE CON PARAMETERS

"""

In [ ]:
# Dudas: cómo crear el Index desde python en vez de desde MongoDB compass?
#db.selected.createIndex( { "position" : "2dsphere" } )

In [ ]:
#query alberto con mis propios criterios: founded year 2003??

In [ ]:
# para luego pasarlo a Tableau!!!!! (incluir el centro en la lista)

"""
m = df_filtered[["category_code","lat","long","name"]]
m = m.append({
    "category_code":"center",
    "lat":fifth_avenue[0],
    "long":fifth_avenue[1],
    "name":"CENTER"
},ignore_index=True)
display(m.head(3))

m.to_json('filtered_data.json', orient="records")

"""

In [ ]:
# poner tb cómo se carga el csv de crunchbase a mongo xt



## TODO's

You recently created a new company in the `GAMING industry`. The company will have the following scheme:

- 20 Designers
- 5 UI/UX Engineers
- 10 Frontend Developers
- 15 Data Engineers
- 5 Backend Developers
- 20 Account Managers
- 1 Maintenance guy that loves basketball
- 10 Executives
- 1 CEO/President

As a data engineer you have asked all the employees to show their preferences on where to place the new office.
Your goal is to place the **new company offices** in the best place for the company to grow.
You have to found a place that more or less covers all the following requirements.
Note that **it's impossible to cover all requirements**, so you have to prioritise at your glance.

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- 30% of the company have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
	- la empresa haya conseguido una financiación > 1M
	- empresa que lleve abierta desde antes de 2003
	- más de 10 empleados.
	- empresas del tipo: design, web, software, games_video, mobile, enterprise, analytics, search, network_hosting o photo_video.
- Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
- Account managers need to travel a lot
- All people in the company have between 25 and 40 years, give them some place to go to party.
- Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.
- The CEO is Vegan

## Help

- Geospatial range Queries in MongoDB with `pymongo`
- GeoJSON Point `{ type: "Point", coordinates: [ 40, 5 ] }`
- Create sphere2d index in python: `db.collection.createIndex( { <location field> : "2dsphere" } )`
- Query `$near` operator: https://docs.mongodb.com/manual/reference/operator/query/near/#op._S_near

## How to deliver the project

- You must justify your decision with tableau slides. Provide us the public tableau link inside a README.md
  file at dir `module-2/project-mongodb-geospartial-queries`.
- Provide `lat` and `long` for the new offices.

## Links & Resources

- https://docs.mongodb.com/manual/geospatial-queries/
- https://developers.google.com/maps/documentation/geocoding/intro
- https://data.crunchbase.com/docs
- https://developers.google.com/places/web-service/search
- https://www.youtube.com/watch?v=PtV-ZnwCjT0


Proceso:
https://github.com/albertogcmr/pymongo-project
Jupiter notebook pymongo-geoqueries-$near.ipynb
https://github.com/boyander/datamad-0819/blob/master/others/geospartial%20queriss.ipynb


----------Apuntes importantes----------

Funciones para convertir:

	$1M  ---> 1000000
	$1K  ---> 1000

	$ ----> dollar
	x -----> libra 
	api de conversión de divisas?

API google maps

LOCATIONS
-todas


VALORACIÓN
- 1 CEO/President
	- vegan
- 20 Designers: 
	- design talks and share knowledge
	- nearby companies that also do design
- 20 Account Managers:
	- need to travel a lot
- 15 Data Engineers
- 10 Executives:
	- Starbucks not too far
- 10 Frontend Developers
	- near successful tech startups that have raised at least 1 Million dollars.
	- la empresa haya conseguido una financiación > 1M
	- empresa que lleve abierta desde antes de 2003
	- más de 10 empleados.
	- empresas del tipo: design, web, software, games_video, mobile, enterprise, analytics, search, 
- 5 UI/UX Engineers
- 5 Backend Developers
- 1 Maintenance guy that loves basketball

- 30% of the company have at least 1 child.
network_hosting o photo_video.
- All people in the company have between 25 and 40 years, give them some place to go to party.
- Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.

Input: le metes una localización y te dice la valoración del sitio para esta empresa en concreto
	---> esto yo para elegir el mejor sitio para mi empresa en concreto
	---> y si me salto este paso?
Input: le metes unos inputs y te dice cuál es la mejor localización para tu empresa
	---> y hago solamente este paso, metiendo mis datos?
	---> creo que lo puedo hacer así
	---> según la importancia que tiene para ti?
	---> con lo de los inputs que me contó jesus?

	PARA STARTUPS
	--->> near companies: (DATABASE)
		--->> raised money (at least $1M)
		--->> sector: design, web, software, games_video, mobile, enterprise, analytics, search
	--->> far from companies: (DATABASE) -- pospuesto
		--->> companies more than 10 years
	--->> close services (schools, supermarket..) (GOOGLE API)
        --->> restaurants/ coffee shops (GOOGLE API)
	--->> events (MEETUP, FORSQUARE, YELP, EVENTBRITE...)




Function gaoling que te enriquece tu base de datos con lo que le vayas preguntando según una ubicación (explicado en el documento


